In [68]:
import pandas as pd
import numpy as np
import datetime as dt
import openpyxl as op

### Read in data and template

In [39]:
file_name = 'Case_Study_Data_AIS.xlsx'

port = pd.read_excel(file_name, sheet_name=2)
sofr = pd.read_excel(file_name, sheet_name=3)

In [73]:
template_name = 'AnalyticsTemplate.xlsx'

wb = op.load_workbook(template_name)
ws = wb.active

In [41]:
port.head()

,Portfolio,Primary_Asset_ID,Asset_Type,Coupon_Type,Rating,Industry,Market_Value,OAS,Yield,Duration
0,Ares,CINAVYTI,Loan,Floating,BB,Media,703181,317.0,4.17,0.115
1,Ares,DUHVUIVB,Bond,Fixed,BB-,Services,955601,387.0,5.87,1.023
2,Ares,BKJJDAID,Bond,Fixed,CCC,Healthcare,732398,519.0,6.19,6.176
3,Ares,JKSOSIKI,Bond,Fixed,BB-,Consumer Goods,1885559,588.0,6.88,6.295
4,Ares,HUIYGUHC,Loan,Floating,B,Financial Services,956697,485.0,6.85,0.236


### Create generic function to calculate market value weighted metrics on portfolio segments

In [42]:
def mvwa(df, analytics): 
    """add docstring"""

    df['Weight'] = df['Market_Value']/df['Market_Value'].sum()
    weighted_analytics = df['Weight'] * df[analytics]
    return weighted_analytics.sum()

In [86]:
port_totl = port.copy()
port_loan = port[port['Asset_Type'] == 'Loan'].copy()
port_bond = port[port['Asset_Type'] == 'Bond'].copy()
port_fixed = port[port['Coupon_Type'] == 'Fixed'].copy()
port_float = port[port['Coupon_Type'] == 'Floating'].copy()

### Loop through metrics and portfolio segment for scalability

In [106]:
port_list = [port_totl, port_bond, port_loan]
analytics_list = ['Yield', 'OAS']

port_list2 = [port_totl, port_fixed, port_float]
analytics_list2 = ['Duration']

In [111]:
num_asset = port_totl.Primary_Asset_ID.value_counts().values.sum()

list2 =[num_asset]

for a in analytics_list: 
    for p in port_list:
        result = mvwa(p, a)
        list2.extend([result])

for a in analytics_list2: 
    for p in port_list2:
        result = mvwa(p, a)
        list2.extend([result])
        
market_value = port_totl.Market_Value.sum()
list2.extend([market_value])

In [207]:
isinstance([1,2], list)

True

### Write out list storing results to the template

In [115]:
for r in ws.iter_rows(min_row=3, min_col=3, max_row=13, max_col=3):
    for cell in r: 
        cell.value = list2[cell.row - 3]
        
wb.save(template_name)

### Get market value percentage, and get template values to be filled out

In [ ]:
def get_mv_percent(port, category):
    """add docstring"""

    mv_percent = port.groupby([category]).agg({'Market_Value': np.sum})/port.Market_Value.sum()
    mv_percent = mv_percent.reset_index()
    mv_percent[category] = mv_percent[category].astype(str)
    
    return mv_percent

In [194]:
def get_template_val(st_row, ed_row, col, category): 
    vals = []
    for row in ws.iter_rows(min_row=st_row, min_col=col, max_row=ed_row, max_col=col):
        for cell in row:
            vals.extend([cell.value])
    df = pd.DataFrame({category: vals})
    df[category]=df[category].astype(str)
    
    return df 

### Calculate for rating

In [200]:
mv_rating = get_mv_percent(port_totl, 'Rating')
mv_rating

,Rating,Market_Value
0,B,0.201154
1,B+,0.143007
2,B-,0.072982
3,BB,0.049749
4,BB+,0.136047
5,BB-,0.199815
6,C,0.010646
7,CCC,0.114753
8,CCC+,0.071846


In [201]:
df_rating = get_template_val(3, 25, 5,'Rating')
df_rating = df_rating.merge(mv_rating, on="Rating", how='left')
df_rating = df_rating.fillna(0)
df_rating

,Rating,Market_Value
0,AAA,0.000000
1,AA+,0.000000
2,AA,0.000000
3,AA-,0.000000
4,A+,0.000000
5,A,0.000000
6,A-,0.000000
7,BBB+,0.000000
8,BBB,0.000000
9,BBB-,0.000000


### Write out to template

In [168]:
for r in ws.iter_rows(min_row=3, min_col=6, max_row=25, max_col=6):
    for cell in r: 
        cell.value = df_rating.Market_Value[cell.row - 3]
        
wb.save(template_name)

### Get market value percentage by Industry

In [177]:
mv_industry = get_mv_percent(port_totl, 'Industry')
mv_industry

,Industry,Market_Value
0,Automotive,0.091238
1,Basic Industry,0.109781
2,Capital Goods,0.064788
3,Consumer Goods,0.027810
4,Energy,0.048622
5,Financial Services,0.057306
6,Healthcare,0.089082
7,Insurance,0.051579
8,Leisure,0.026133
9,Media,0.054613


In [202]:
df_industry = get_template_val(st_row=32, ed_row=49, col=5, category ='Industry')
df_industry = df_industry.merge(mv_industry, on='Industry', how="left") 
df_industry = df_industry.fillna(0)
df_industry

,Industry,Market_Value
0,Automotive,0.091238
1,Banking,0.000000
2,Basic Industry,0.109781
3,Capital Goods,0.064788
4,Consumer Goods,0.027810
5,Energy,0.048622
6,Financial Services,0.057306
7,Healthcare,0.089082
8,Insurance,0.051579
9,Leisure,0.026133


### Write out to template 

In [206]:
for r in ws.iter_rows(min_row=32, min_col=6, max_row=49, max_col=6):
    for cell in r: 
        cell.value = df_industry.Market_Value[cell.row - 32]
        
wb.save(template_name)